In [9]:
#Oppgave 2
import requests
import pandas as pd
import json
from pandasql import sqldf

#
API_URL = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
headers = {"User-Agent": "MyWeatherApp/1.0 (contact@example.com)", "Accept": "application/json"}
params = {"lat": 59.91, "lon": 10.75}  # Oslo koordinater

response = requests.get(API_URL, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    with open("../data/weather_data.json", "w") as f:
        json.dump(data, f, indent=4)
    print("Data hentet og lagret som JSON")
else:
    print("Feil ved henting av data! Statuskode:", response.status_code)
    exit()

# 
timeseries = data["properties"]["timeseries"]
rows = []

for entry in timeseries:
    time = entry["time"]
    details = entry["data"]["instant"]["details"]
    rows.append({"Tidspunkt": time, **details})

df = pd.DataFrame(rows)

#
column_mapping = {
    "air_pressure_at_sea_level": "Lufttrykk ved havnivå",
    "air_temperature": "Lufttemperatur",
    "wind_speed": "Vindhastighet",
    "wind_from_direction": "Vindretning",
    "relative_humidity": "Relativ fuktighet",
    "cloud_area_fraction": "Skydekke"
}
df.rename(columns=column_mapping, inplace=True)

df.to_csv("../data/weather_data.csv", index=False)
print("Data lagret som CSV")

#
source_info = {
    "Kilde": "MET Norway",
    "Pålitelighet": "Høy",
    "Tilgjengelighet": "Åpen tilgang, krever User-Agent header",
    "Format": "JSON",
    "Oppdateringsfrekvens": "Løpende oppdateringer",
}
print("Vurdering av kilde:")
for key, value in source_info.items():
    print(f"{key}: {value}")

#
pysqldf = lambda q: sqldf(q, globals())
query = """
SELECT Tidspunkt, Lufttemperatur, Vindhastighet
FROM df
WHERE Lufttemperatur IS NOT NULL
ORDER BY Tidspunkt ASC
LIMIT 5;
"""
result = pysqldf(query)
print("\nEksempel på analyserte data:")
print(result)


Data hentet og lagret som JSON
Data lagret som CSV
Vurdering av kilde:
Kilde: MET Norway
Pålitelighet: Høy
Tilgjengelighet: Åpen tilgang, krever User-Agent header
Format: JSON
Oppdateringsfrekvens: Løpende oppdateringer

Eksempel på analyserte data:
              Tidspunkt  Lufttemperatur  Vindhastighet
0  2025-03-26T12:00:00Z            10.3            2.0
1  2025-03-26T13:00:00Z            11.6            2.0
2  2025-03-26T14:00:00Z            12.7            3.0
3  2025-03-26T15:00:00Z            12.6            3.7
4  2025-03-26T16:00:00Z            11.4            3.9


In [ ]:
#Oppgave 3
# 
def clean_data(df):
    df.dropna(inplace=True)  # Fjerner rader med manglende verdier
    return df

df_cleaned = clean_data(df)
print("\nData etter håndtering av manglende verdier:")
print(df_cleaned.head())

#List comprenhensions 
df_cleaned["Tidspunkt"] = [t.replace("T", " ") for t in df_cleaned["Tidspunkt"]]
print("\nTidspunkt formatert:")
print(df_cleaned["Tidspunkt"].head())

# 
query = """
SELECT Tidspunkt, Lufttemperatur, Vindhastighet
FROM df_cleaned
WHERE Lufttemperatur > 0
ORDER BY Vindhastighet DESC
LIMIT 5;
"""
result = pysqldf(query)
print("\nEksempel på filtrerte data:")
print(result)

#
print("\nForventede uregelmessigheter og håndtering:")
print("- Manglende verdier: Løst med dropna().")
print("- Uregelmessige tidspunkter: Formatert med list comprehension.")
print("- Ekstreme verdier: Kan analyseres med ytterligere filtrering.")



Data etter håndtering av manglende verdier:
              Tidspunkt  Lufttrykk ved havnivå  Lufttemperatur  Skydekke  \
0  2025-03-26 12:00:00Z                 1015.9            10.3      99.5   
1  2025-03-26 13:00:00Z                 1015.5            11.6     100.0   
2  2025-03-26 14:00:00Z                 1015.3            12.7     100.0   
3  2025-03-26 15:00:00Z                 1015.0            12.6      99.7   
4  2025-03-26 16:00:00Z                 1015.0            11.4      97.1   

   Relativ fuktighet  Vindretning  Vindhastighet  
0               58.4        187.4            2.0  
1               55.4        195.3            2.0  
2               53.2        199.4            3.0  
3               56.1        180.1            3.7  
4               61.6        175.7            3.9  

Tidspunkt formatert:
0    2025-03-26 12:00:00Z
1    2025-03-26 13:00:00Z
2    2025-03-26 14:00:00Z
3    2025-03-26 15:00:00Z
4    2025-03-26 16:00:00Z
Name: Tidspunkt, dtype: object

Eksempel